In [ ]:
using Distributed
addprocs(12)

@everywhere include("problem.jl")
@everywhere include("../fileIO.jl")

@everywhere using LinearAlgebra, Random, ProgressMeter

# 4D Cartpole System
Implementation of cart-pole system with contacts from _Warm Start of Mixed-Integer Programs for Model Predictive Control of Hybrid Systems_, Marcucci & Tedrake (2019)

In [ ]:
@everywhere function make_dataset(jj;Ndata=5000)
    Random.seed!(jj)
    
    # params
    n,m = 4, 3
    N = 11
    Q = diagm([10.;10;10;10])
    R = diagm([1.;0;0])
    l = 1.
    mc = 1.
    mp = 1.
    g = 10.
    dh = 0.05
    kappa,nu = 100., 30. # contact parameters

    # state bounds
    dist = 0.5
    x_max = [dist;pi/8;2;1]
    x_min = -x_max

    delta_min = [-x_max[1] + l*x_min[2] - dist;
                  x_min[1] - l*x_max[2] - dist]
    ddelta_min= [-x_max[3] + l*x_min[4];
                x_min[3] - l*x_max[4]]

    delta_max = [-x_min[1] + l*x_max[2] - dist;
                  x_max[1] - l*x_min[2] - dist]
    ddelta_max = [-x_min[3] + l*x_max[4];
                  x_max[3] - l*x_min[4]]

    # control bounds
    uc_min = -2.
    uc_max = -uc_min
    sc_min = kappa*delta_min + nu*ddelta_min
    sc_max = kappa*delta_max + nu*ddelta_max

    # dynamics
    A = [zeros(2,2) Matrix(I,2,2); zeros(2,4)]
    A[3,2] = g*mp/mc
    A[4,2] = g*(mc+mp)/(l*mc)
    B = zeros(n,m)
    B[3,1] = 1/mc
    B[4,:] = [1/(l*mc) -1/(l*mp) 1/(l*mp)]
    Ak = Matrix(I,n,n) + dh*A
    Bk = dh*B

    params = Dict("n"=>n, "m"=>m, "dh"=>dh, "kappa"=>kappa, "nu"=>nu, 
        "mc"=>mc, "mp"=>mp, "g"=>g, "l"=>l, "dist"=>dist)

    UB = upperBoundCartpoleProb()[1]
    probdata = Dict("n"=>n, "m"=>m, "dh"=>dh, "N"=>N, "Q"=>Q, "R"=>R,
                    "l"=>l, "mc"=>mc, "mp"=>mp, "g"=>g, "kappa"=>kappa, "nu"=>nu,"dist"=>dist,
                    "x_min"=>x_min, "x_max"=>x_max, "uc_min"=>uc_min, "uc_max"=>uc_max, "sc_min"=>sc_min, "sc_max"=>sc_max,
                    "delta_min"=>delta_min, "delta_max"=>delta_max,"ddelta_min"=>ddelta_min, "ddelta_max"=>ddelta_max,
                    "Ak"=>Ak, "Bk"=>Bk, "params"=>params, "UB"=>UB);

    # Problem
    X0 = []
    Xg = []
    X = []
    Y = []
    Uc = []
    Sc = []
    J = Float64[]
    solution_times = Float64[]
    node_counts = Int[]

    ii = 0
    while ii < Ndata
        x0 = 0.5*(x_min+x_max) + (rand(n).-0.5).*(x_max-x_min)
        xg = zeros(n)

        # Solve Problem        
        x,uc,sc,y,cost,solve_time,node_count,solver_success = 
            solveCartpoleProb(Ak,Bk,Q,R,N,uc_min,uc_max,sc_min,sc_max,x_min,x_max,
            delta_min,delta_max,ddelta_min,ddelta_max,probdata,x0,xg)

        # Save all the data
        if solver_success
            push!(X0, x0)
            push!(Xg, xg)
            push!(X, x)
            push!(Uc, uc)
            push!(Sc, sc)
            push!(Y, y)
            push!(J, cost)
            push!(solution_times, solve_time)
            push!(node_counts, node_count)

            ii += 1
        end
    end
    
    base = "data/testdata"
    filename = string(base, string(jj), ".jld")

    training_data = Dict("X0"=>X0, "Xg"=>Xg, "X"=>X, "Y"=>Y,
        "Uc"=>Uc, "Sc"=>Sc, "J"=>J, 
        "solve_time"=>solution_times, "node_count"=>node_counts,
        "prob"=>probdata)
    writeTrainingData(filename, training_data)
    return
end

In [ ]:
Nsets = 20
Ndata = 5000
dir_idxs = [ii for ii in 1:Nsets]
time_parallel = @elapsed results = @showprogress pmap(make_dataset, dir_idxs)    